In [5]:
import boto3
from dotenv import load_dotenv
import os
import csv

load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = os.getenv("AWS_DEFAULT_REGION")

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_default_region
)

dynamodb = boto3.client('dynamodb')

table_name = 'User-7kkcm5dn2rb77hst5nh7gbdisa-staging'

with open('dynamodb_users.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    last_evaluated_key = None

    all_columns = set()

    while True:
        if last_evaluated_key:
            response = dynamodb.scan(
                TableName=table_name,
                ExclusiveStartKey=last_evaluated_key
            )
        else:
            response = dynamodb.scan(TableName=table_name)

        for item in response['Items']:
            for key in item.keys():
                all_columns.add(key)

        if file.tell() == 0:  
            writer.writerow(sorted(all_columns))

        for item in response['Items']:
            row = []
            for column in sorted(all_columns):  
                value = item.get(column, {}).get('S', '') 
                if not value:  
                    value = item.get(column, {}).get('N', '')  
                row.append(value)
            
            writer.writerow(row)

        if 'LastEvaluatedKey' in response:
            last_evaluated_key = response['LastEvaluatedKey']
        else:
            break

print("Datos exportados a dynamodb_users.csv")



Datos exportados a dynamodb_users.csv
